In [9]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {'S': 0, 'T': 1, 'L': 2,'O': 3,'I': 4}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])

X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
#         self.conv1 = nn.Conv2d(1, 3, 3, padding=1)
#         self.mp1 = nn.MaxPool2d(3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(3, 3, 3, padding=1)
#         self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
#         self.fc = nn.Linear(27, 5)

        self.conv1 = nn.Conv2d(1, 2, 2)
        self.conv2 = nn.Conv2d(2, 10, 2)
        self.fc = nn.Linear(10, 5)


    def forward(self, x):
        in_size = x.size(0)
#         x = self.mp1(F.relu(self.conv1(x)))
#         x = self.mp2(F.relu(self.conv2(x)))       
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)


def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print(get_n_params(ConvNet()))

# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(dataset=dataset,
                          batch_size=200,
                          shuffle=True,
                          num_workers=2)
    
    for _, data in enumerate(train_loader, 0):
#     for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = X_train[i], torch.tensor(Y_train[i])
        inputs, labels = data
#         inputs = inputs.view(256, 1, 3, 3)
#         for i in range(len(labels)):
#             x = inputs[i].view(1, 1, 3, 3)

        inputs = inputs.view(200,1,3,3)
        labels = labels.long()
        optimizer.zero_grad()
        outputs = model(inputs)
#         print(outputs)
#         print(labels[i])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 100 == 99:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
#                 labels = labels.squeeze(0).max(0)[1]
                output = model(images)
                val, predicted = output.data.squeeze(0).max(0)
                correct += int(predicted.float()==labels.float())
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))

        results[epoch+1] = [running_loss, accuracy]

155
Epoch: 100, Loss: 3.916
Accuracy of the network: 50.50 %
Epoch: 200, Loss: 3.881
Accuracy of the network: 51.00 %
Epoch: 300, Loss: 3.884
Accuracy of the network: 49.50 %
Epoch: 400, Loss: 3.890
Accuracy of the network: 50.00 %
Epoch: 500, Loss: 3.878
Accuracy of the network: 52.00 %
Epoch: 600, Loss: 3.886
Accuracy of the network: 51.00 %
Epoch: 700, Loss: 3.883
Accuracy of the network: 51.00 %
Epoch: 800, Loss: 3.881
Accuracy of the network: 49.00 %
Epoch: 900, Loss: 3.882
Accuracy of the network: 49.50 %
Epoch: 1000, Loss: 3.876
Accuracy of the network: 48.50 %


In [ ]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
    'T': [0,0,0,1,0],
    'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
    'I': [1,0,0,0,0]
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3, padding=1)
        self.mp1 = nn.MaxPool2d(3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(3, 3, 3, padding=1)
        self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
        self.fc = nn.Linear(27, 5)        
        
#         self.conv1 = nn.Conv2d(1, 5, 2)
#         self.conv2 = nn.Conv2d(5, 10, 2)
#         self.fc = nn.Linear(10, 5)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = self.mp2(F.relu(self.conv2(x)))       
#         x = self.conv1(x)
#         x = self.conv2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()

# optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)


def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

# get_n_params(ConvNet())

# Training
results = {} #{iter: [loss, accuracy]}
for epoch in range(10000):  # loop over the dataset multiple times

    running_loss = 0.0
    dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(dataset=dataset,
                          batch_size=200,
                          shuffle=True,
                          num_workers=2)
    
    for _, data in enumerate(train_loader, 0):

        inputs, labels = data

        inputs = inputs.view(200,1,3,3)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 100 == 99:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
                labels = labels.max(0)[1]
                output = model(images)
#                 print(output, labels)
                _, predicted = output.data.squeeze(0).max(0)
        
                correct += int(predicted.float()==labels.float())
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))

        results[epoch+1] = [running_loss, accuracy]

Epoch: 100, Loss: 0.031
Accuracy of the network: 100.00 %
Epoch: 200, Loss: 0.025
Accuracy of the network: 100.00 %
Epoch: 300, Loss: 0.018
Accuracy of the network: 100.00 %
Epoch: 400, Loss: 0.017
Accuracy of the network: 100.00 %
Epoch: 500, Loss: 0.013
Accuracy of the network: 100.00 %
Epoch: 600, Loss: 0.010
Accuracy of the network: 100.00 %
Epoch: 700, Loss: 0.011
Accuracy of the network: 100.00 %
Epoch: 800, Loss: 0.011
Accuracy of the network: 100.00 %
Epoch: 900, Loss: 0.010
Accuracy of the network: 100.00 %
Epoch: 1000, Loss: 0.011
Accuracy of the network: 100.00 %
Epoch: 1100, Loss: 0.009
Accuracy of the network: 100.00 %
Epoch: 1200, Loss: 0.009
Accuracy of the network: 100.00 %
Epoch: 1300, Loss: 0.010
Accuracy of the network: 100.00 %
Epoch: 1400, Loss: 0.009
Accuracy of the network: 100.00 %
Epoch: 1500, Loss: 0.008
Accuracy of the network: 100.00 %
Epoch: 1600, Loss: 0.008
Accuracy of the network: 100.00 %
Epoch: 1700, Loss: 0.008
Accuracy of the network: 100.00 %
Epoch: